In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

In [2]:
import nitime
import nitime.analysis as nta
import nitime.fmri.io as io

In [40]:
seedFile = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/regMeants/arima_residu_ts.txt'

seed_ts_array = np.loadtxt(seedFile)
#print seed_ts_array

from nitime.timeseries import TimeSeries
ts = TimeSeries(seed_ts_array, sampling_interval=2.0)
print ts.data

[ -2.22658507e-03   1.96420073e+00   1.92572899e+00  -3.71609916e-03
   2.27342658e+00   8.86673625e-01   3.02894958e-01   1.97555637e+00
  -1.94825915e-01   4.30060947e-01   1.69298431e+00   2.85184416e+00
   2.48645310e+00  -2.42304416e+00  -1.07544962e+00  -9.00403737e-01
  -3.12118713e+00   8.59856138e-01  -4.28875315e+00  -8.09222956e+00
  -9.29063535e-01  -6.68012318e-01  -3.22326002e+00  -5.43267394e+00
  -4.09624486e+00   2.50935831e+00  -9.40407908e-01  -4.53184258e+00
  -1.08141672e+00   5.44823591e-01   3.09837976e+00   2.15491274e-01
  -3.77181284e+00   6.29105842e-01   2.14086325e+00   4.90598933e-01
   1.36135056e+00   1.37700904e+00   5.59982037e-01   1.10489074e+00
   2.46440385e+00   1.26070708e+00  -9.37960655e-01   5.38778329e-01
   3.41878201e+00   3.10949465e+00   1.54615178e+00   8.32029575e-01
  -1.32914533e+00  -5.22107635e-01   1.41428459e+00  -7.98545217e-01
   8.06791950e-01   5.30421652e-01  -4.23840332e+00  -8.28038608e-01
   2.10814827e+00   3.93404787e-02

In [36]:
from nitime.timeseries import TimeSeries
ts = TimeSeries([1,2,3],sampling_interval=2.0)
print ts.data

[1 2 3]


In [12]:
TR = 2
f_lb = 0.01
f_ub = 0.1

fmri_file = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/arima/arimaResidus/_report/arima_residu.nii.gz'
fmri_data=nib.load(fmri_file) 

In [13]:
volume_shape = fmri_data.shape[:-1]
print volume_shape
coords = list(np.ndindex(volume_shape))

(91, 109, 91)


In [25]:
n_seeds = 3

# Choose n_seeds random voxels to be the seed voxels
seeds = np.random.randint(0, len(coords), n_seeds)
coords_seeds = np.array(coords)[seeds].T
print coords_seeds
coords_target = np.array(coords).T

[[ 0 33 81]
 [58 59 59]
 [75 79 41]]


In [26]:
# Make the seed time series:
time_series_seed = io.time_series_from_file(fmri_file,
                                coords_seeds,
                                TR=TR,
                                normalize='percent',
                                filter=dict(lb=f_lb,
                                            ub=f_ub,
                                            method='boxcar'))

# Make the target time series:
time_series_target = io.time_series_from_file(fmri_file,
                                          coords_target,
                                          TR=TR,
                                          normalize='percent',
                                          filter=dict(lb=f_lb,
                                                      ub=f_ub,
                                                    method='boxcar'))

In [27]:
A = nta.SeedCoherenceAnalyzer(time_series_seed, time_series_target,
                            method=dict(NFFT=20))
B = nta.SeedCorrelationAnalyzer(time_series_seed, time_series_target)

In [28]:
freq_idx = np.where((A.frequencies > f_lb) * (A.frequencies < f_ub))[0]

In [29]:
cor = []
coh = []

for this_seed in range(n_seeds):
    # Extract the coherence and average across these frequency bands:
    coh.append(np.mean(A.coherence[this_seed][:, freq_idx], -1))  # Averaging on the
                                                                 # last dimension

    cor.append(B.corrcoef[this_seed])  # No need to do any additional
                                       # computation

In [24]:
coh

[array([  1.,  nan,  nan]),
 array([ nan,  nan,  nan]),
 array([ nan,  nan,  nan])]